In [2]:
mutable struct Account
    account_number::String
    balance::Float64
    date_opened::Date
end

account_number(a::Account)=a.account_number
balance(a::Account)=a.balance
date_opened(a::Account)=a.date_opened

function deposit!(a::Account,amount::Real)
    a.balance+=amount
    return a.balance
end

function withdraw!(a::Account,amount::Real)
    a.balance-=amount
    return a.balance
end

function transfer!(from::Account,to::Account,amount::Real)
    withdraw!(from,amount)
    deposit(to,amount)
    return amount
end


LoadError: UndefVarError: Date not defined

In [1]:
struct SavingAccount
    acct::Account
    interest_rate::Float64
    #constructor
    SavingAccount(account_number,balance,date_opened)=new(Account(account_number,balance,date_opened),interest_rate)
end

using Lazy: @forward

LoadError: UndefVarError: Account not defined

In [1]:
using SimpleTraits
abstract type Asset end
abstract type Property <:Asset end
abstract type Investment <:Asset end
abstract type Cash <: Asset end
abstract type House <:Property end
abstract type Apartment <:Property end
abstract type FixedIncome <:Investment end
abstract type Equity <: Investment end


In [2]:
struct Stock<:Equity 
    symbol
    name
end
struct Pension <:Investment
    value
end


struct Money<:Cash
    currency 
    amount
    value
end

struct Suica<:Cash
    currency
    amount
    value
end



In [4]:
@traitdef IsLiquid{T}
@traitimpl IsLiquid{Cash}
@traitimpl IsLiquid{Investment}


@traitfn marketprice(x::::IsLiquid)=x.value


marketprice (generic function with 2 methods)

In [1]:
pen=Pension(1900)
@show marketprice(pen)
money=Money(100,100,100)
@show marketprice(money)

LoadError: UndefVarError: Pension not defined

In [7]:
using  Dates
abstract type Asset end
abstract type Investment <: Asset end
abstract type Equity <: Investment end

struct Stock <: Equity
    symbol::String
    name::String
end

abstract type Trade end
@enum LongShort Long Short

struct StockTrade <: Trade
    type::LongShort
    stock::Stock
    quantity::Int
    price::Float64
end

@enum CallPut Call Put
struct StockOption <:Equity
        symbol::String
        type::CallPut
        strike::Float64
        expiration::Date
end
struct StockoptionTrade<:Trade
    type::LongShort
    option::StockOption
    quantity::Int
    price::Float64
end
            
            


In [18]:
#stocktrade and stockoption trade are very similar ,so 
#make it more simple with parametric types

struct SingleTrade{T<:Investment} <: Trade
    type::LongShort
    instrument::T
    quantity::Int
    price::Float64
end
    
@show stock=Stock("aapl","apple inc")
@show option=StockOption("aapl",Call,200,Date(2020,4,4))
@show a=SingleTrade(Long,stock,100,188.0)
@show b=SingleTrade(Long,option,100,3.6)

    
function sign(t::SingleTrade)
    return t.type==Long ? 1 : -1
end
function payment(t::SingleTrade)
    return sign(t)*t.quantity*t.price
end
function payment(t::SingleTrade{StockOption}) 
    return sign(t)*t.quantity*100*t.price
end
@show payment(SingleTrade(Long,option,1,3.5))   

stock = Stock("aapl", "apple inc") = Stock("aapl", "apple inc")
option = StockOption("aapl", Call, 200, Date(2020, 4, 4)) = StockOption("aapl", Call, 200.0, Date("2020-04-04"))
a = SingleTrade(Long, stock, 100, 188.0) = SingleTrade{Stock}(Long, Stock("aapl", "apple inc"), 100, 188.0)
b = SingleTrade(Long, option, 100, 3.6) = SingleTrade{StockOption}(Long, StockOption("aapl", Call, 200.0, Date("2020-04-04")), 100, 3.6)
payment(SingleTrade(Long, option, 1, 3.5)) = 350.0


350.0

In [19]:
struct PairTrade{T<:Investment,S<:Investment}<:Trade
    leg1::SingleTrade{T}
    leg2::SingleTrade{S}
end

payment(t::PairTrade)=payment(t.leg1)+payment(t.leg2)

payment (generic function with 3 methods)